<a href="https://colab.research.google.com/github/qinalan10/cs598-patient-phenotyping/blob/master/Code/DLH_Patient_Phenotyping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# importing libaries
from google.colab import files
from google.colab import drive
import pandas as pd 
from torchsummary import summary
import torch
from torch.utils.data import Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import re 
import numpy as np 
from sklearn.metrics import classification_report
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score   
from torchsummary import summary
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc, RocCurveDisplay
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA A100-SXM4-40GB


In [16]:
path = '/content/drive/MyDrive/Data/NOTEEVENTS.csv'
annotations_path = "/content/drive/MyDrive/Data/annotations.csv"
final_file_path = "/content/drive/MyDrive/Data/final_file.csv"

In [18]:
dataframe = pd.read_csv(final_file_path)
dataframe.drop('Unnamed: 0', axis=1, inplace=True)

In [19]:
class HealthcareTextDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.text_data = self.data['TEXT']
        self.labels = torch.tensor(np.array(self.data.iloc[:, :-1]), dtype = torch.float, device=device)  # assuming the labels are in the first 14 columns
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.text_data.iloc[idx]
        labels = self.labels[idx]
        sample = {'text': text, 'labels': labels}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [20]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))
stp_wrds = stopwords.words('english')
stp_wrds.extend(["uncle", "grandmother", "grandfather"])
print(stp_wrds)
#stop_words = set(stopwords.words('english'))
stop_words = set(stp_wrds)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
tokenizer = get_tokenizer('basic_english')

import string
# Define a custom data transformation function
def transform_fn(sample):
    text = sample['text']
    labels = sample['labels']

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'\d+', '', text) # maybe don't remove, could be important 
    
    # Tokenize text data
    text = tokenizer(text)

    filtered_words  = []
    for word in text:
      if word not in stop_words:
        filtered_words.append(word)
    text = filtered_words

    # Convert labels to PyTorch tensor
    return {'text': text, 'labels': labels}

In [22]:
dataset = HealthcareTextDataset(dataframe, transform_fn)
print(dataset[1])

{'text': ['admission', 'date', 'discharge', 'date', 'date', 'birth', 'sex', 'service', 'medicine', 'allergies', 'ceclor', 'compazine', 'attendingknown', 'firstname', 'chief', 'complaint', 'abdominal', 'pain', 'major', 'surgical', 'invasive', 'procedure', 'none', 'history', 'present', 'illness', 'year', 'old', 'male', 'pmh', 'barretts', 'esophagous', 'sp', 'nissen', 'fundoplication', 'emergent', 'splenectomy', 'sp', 'snow', 'boarding', 'accident', 'metastatic', 'soft', 'tissue', 'fibrosarcoma', 'sp', 'small', 'bowel', 'resection', 'hospital', 'started', 'chemo', 'month', 'transition', 'temador', 'chemotherapy', 'last', 'dose', 'first', 'week', 'month', 'presents', 'fevers', 'crampy', 'abdominal', 'pain', 'watery', 'nonbloody', 'diarrhea', 'twothree', 'weeks', 'prior', 'admission', 'patient', 'originally', 'admitted', 'hospital', 'location', 'un', 'fevers', 'abdominal', 'pain', 'watery', 'diarrhea', 'found', 'c', 'diff', 'positive', 'initially', 'treated', 'po', 'vanco', 'iv', 'flagyl', 

In [23]:
def collate_fn(batch):
    texts = [item['text'] for item in batch]  # Extract the text data from the batch
    labels = [item['labels'] for item in batch]  # Extract the labels from the batch

    # Find the maximum length of text sequences in the batch
    max_len = max([len(text) for text in texts])

    # Pad the text sequences with a padding token (e.g., '<pad>') to the maximum length
    padded_texts = []
    for text in texts:
      padded_text = text + ['<pad>'] * (max_len - len(text))
      padded_texts.append(padded_text)

    # Convert the labels to a PyTorch tensor
    labels = torch.stack(labels)

    return {'text': padded_texts, 'labels': labels}

In [24]:
# Setting up random seed
torch.manual_seed(456)

# Create train, test and validation split
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  collate_fn=collate_fn)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

Training a Word2Vec Model

In [25]:
from gensim.models import Word2Vec

corpus_processed = []
for i in train_dataset:
  corpus_processed.append(i['text'])

RANDOM_SEED = 23432098
VEC_SIZE = 300
MIN_COUNT = 5
sg = 0 # 0 is CBOW, 1 is skip gram
w2v_model = Word2Vec(sentences=corpus_processed, 
                     vector_size= VEC_SIZE, 
                     min_count = MIN_COUNT, 
                     workers=1, 
                     seed = RANDOM_SEED, 
                     sg=sg, 
                     epochs=15,
                     window=10,
                     negative=10)


In [26]:
kv = w2v_model.wv
kv.vectors
#then convert to kv.vectors to tensor
embeddings = torch.tensor(kv.vectors, device=device)
print(embeddings.shape)
print(embeddings)

[('morbid', 0.8462918996810913), ('osa', 0.7151551842689514), ('migraines', 0.6937200427055359), ('asthma', 0.6794718503952026), ('apnea', 0.6638763546943665), ('oa', 0.6443200707435608), ('osteoarthritis', 0.6354600191116333), ('hypoventilation', 0.6208170056343079), ('osteoporosis', 0.6116981506347656), ('cataracts', 0.609263002872467)]
torch.Size([10315, 300])
tensor([[-1.6004e+00, -4.0572e-02,  1.4913e+00,  ...,  5.6209e-01,
         -1.4693e+00,  6.5162e-01],
        [-8.4426e-01, -3.9578e-01,  2.7766e-01,  ..., -6.4214e-02,
          3.8522e-01,  8.1983e-01],
        [ 8.6164e-01, -4.8736e-02,  2.9800e-01,  ...,  5.4254e-01,
          4.4704e-01,  1.1246e+00],
        ...,
        [-6.1528e-03, -4.5394e-02, -7.8702e-02,  ...,  6.1781e-02,
         -2.5810e-02, -6.7572e-02],
        [-1.3252e-01, -3.2614e-02,  1.5242e-01,  ..., -1.7464e-01,
          9.4650e-02, -5.6039e-03],
        [ 1.0066e-01,  4.4941e-04, -1.7577e-01,  ..., -1.9175e-01,
          1.4803e-01, -6.0401e-02]], de

In [27]:
#input: model: word2vec model
#       word: a single word, e.g. 'heart'
#output: list_of_words: a list of words similar to the given word
def similar_word(model, word):
    
    list_of_words = []

    # your code here
    tup_of_words = model.wv.most_similar(word)
    list_of_words = list(map(lambda x: x[0], tup_of_words))
    return list_of_words

In [28]:
word = 'cancer'
assert type(similar_word(w2v_model, word)) is list
print("The words similar to '%s' are: '%s'." % (word, ', '.join(similar_word(w2v_model, word))))

word = 'obesity'
assert type(similar_word(w2v_model, word)) is list
print("The words similar to '%s' are: '%s'." % (word, ', '.join(similar_word(w2v_model, word))))

word = 'blood'
assert type(similar_word(w2v_model, word)) is list
print("The words similar to '%s' are: '%s'." % (word, ', '.join(similar_word(w2v_model, word))))

word = 'heart'
assert type(similar_word(w2v_model, word)) is list
print("The words similar to '%s' are: '%s'." % (word, ', '.join(similar_word(w2v_model, word))))

The words similar to 'cancer' are: 'ca, mets, adenocarcinoma, carcinoma, nsclc, rcc, tumor, xrt, breast, lymphoma'.
The words similar to 'obesity' are: 'morbid, osa, migraines, asthma, apnea, oa, osteoarthritis, hypoventilation, osteoporosis, cataracts'.
The words similar to 'blood' are: 'dropped, support, pcwpmmhg, sent, high, nailbed, urine, yearmonthday, estimated, surveillance'.


In [29]:
w2v_index = kv.index_to_key
print(w2v_index)
print(len(w2v_index))

['mg', 'po', 'name', 'tablet', 'patient', 'blood', 'sig', 'daily', 'hospital', 'one', 'day', 'discharge', 'history', 'last', 'left', 'pm', 'admission', 'right', 'pain', 'pt', 'date', 'namepattern', 'ct', 'medications', 'times', 'normal', 'chest', 'first', 'given', 'also', 'status', 'hours', 'sp', 'course', 'disease', 'every', 'continued', 'needed', 'home', 'stable', 'well', 'capsule', 'started', 'please', 'care', 'md', 'days', 'two', 'per', 'bid', 'negative', 'qh', 'un', 'hct', 'refills', 'x', 'showed', 'prior', 'stitle', 'time', 'tablets', 'wbc', 'medical', 'likely', 'disp', 'dr', 'seen', 'past', 'acute', 'renal', 'pulmonary', 'glucose', 'iv', 'service', 'patients', 'urine', 'release', 'chronic', 'edema', 'noted', 'followup', 'failure', 'due', 'qd', 'l', 'instructions', 'heart', 'prn', 'exam', 'present', 'artery', 'family', 'plt', 'mild', 'known', 'admitted', 'rbc', 'tid', 'without', 'pressure', 'physical', 'insulin', 'units', 'k', 'tube', 'follow', 'lower', 'evidence', 'hr', 'seconda

In [30]:
def word2idx(w2v_index):
    word_to_idx = {'<pad>':0}
    idx = 1
    for word in w2v_index:
        if word not in word_to_idx:
            word_to_idx[word] = idx
            idx += 1
    return word_to_idx

In [31]:
w2v_dictionary = word2idx(w2v_index)
print(w2v_dictionary)

{'<pad>': 0, 'mg': 1, 'po': 2, 'name': 3, 'tablet': 4, 'patient': 5, 'blood': 6, 'sig': 7, 'daily': 8, 'hospital': 9, 'one': 10, 'day': 11, 'discharge': 12, 'history': 13, 'last': 14, 'left': 15, 'pm': 16, 'admission': 17, 'right': 18, 'pain': 19, 'pt': 20, 'date': 21, 'namepattern': 22, 'ct': 23, 'medications': 24, 'times': 25, 'normal': 26, 'chest': 27, 'first': 28, 'given': 29, 'also': 30, 'status': 31, 'hours': 32, 'sp': 33, 'course': 34, 'disease': 35, 'every': 36, 'continued': 37, 'needed': 38, 'home': 39, 'stable': 40, 'well': 41, 'capsule': 42, 'started': 43, 'please': 44, 'care': 45, 'md': 46, 'days': 47, 'two': 48, 'per': 49, 'bid': 50, 'negative': 51, 'qh': 52, 'un': 53, 'hct': 54, 'refills': 55, 'x': 56, 'showed': 57, 'prior': 58, 'stitle': 59, 'time': 60, 'tablets': 61, 'wbc': 62, 'medical': 63, 'likely': 64, 'disp': 65, 'dr': 66, 'seen': 67, 'past': 68, 'acute': 69, 'renal': 70, 'pulmonary': 71, 'glucose': 72, 'iv': 73, 'service': 74, 'patients': 75, 'urine': 76, 'release

In [32]:
idx_0 = torch.zeros(1, 300, device = device)
embeddings = torch.cat((idx_0, embeddings))

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random 

class CNNClassifier(nn.Module):
    def __init__(self, embedding_dim, num_filters, filter_sizes, embeddings, output_size, w2v_dictionary):
        super(CNNClassifier, self).__init__()
        self.dictionary = w2v_dictionary
        self.embedding = nn.Embedding.from_pretrained(embeddings)#(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim, num_filters, filter_sizes[0])
        self.conv2 = nn.Conv1d(embedding_dim, num_filters, filter_sizes[1])
        self.conv3 = nn.Conv1d(embedding_dim, num_filters, filter_sizes[2])
        self.conv4 = nn.Conv1d(embedding_dim, num_filters, filter_sizes[3])
        self.dropout = nn.Dropout(p = 0.1)
        self.fc = nn.Linear(len(filter_sizes) * num_filters, output_size)
        
    def forward(self, x):
      # change to indexes, then to tensor
        z2 = []
        for ls in x:
          z1 = []
          for word in ls:
            if word not in self.dictionary.keys():
              random_key = random.sample(list(self.dictionary.keys()), 1)[0]
              z1.append(self.dictionary[random_key])
            else:
              z1.append(self.dictionary[word])
          z2.append(z1)  

        x = torch.tensor(z2, device = device)
        x = self.embedding(x)
        x = x.permute(0, 2, 1)  # Permute the dimensions for Conv1d (batch_size, embedding_dim, sequence_length)

        x1 = F.relu(self.conv1(x))
        x1 = F.max_pool1d(x1 , x1.size(2)).squeeze(2)

        x2 = F.relu(self.conv2(x))
        x2 = F.max_pool1d(x2 , x2.size(2)).squeeze(2)

        x3 = F.relu(self.conv3(x))
        x3 = F.max_pool1d(x3 , x3.size(2)).squeeze(2)

        x4 = F.relu(self.conv4(x))
        x4 = F.max_pool1d(x4 , x4.size(2)).squeeze(2)

        out = torch.cat((x1, x2, x3, x4), 1)
        out = self.dropout(out)
        out = self.fc(out)

        out = torch.sigmoid(out)

        return out

In [34]:
#vocab_size = len(vocab)
embedding_dim = 300
num_filters = 256
filter_sizes = [1, 2, 3, 5]
output_size = 14

print(len(filter_sizes) * num_filters)
model = CNNClassifier(embedding_dim, num_filters, filter_sizes, embeddings, output_size, w2v_dictionary)

model.to(device)

summary(model, input_size=(5, 1), batch_size=-1)

1024
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Embedding-1               [-1, 5, 300]       3,094,800
            Conv1d-2               [-1, 256, 5]          77,056
            Conv1d-3               [-1, 256, 4]         153,856
            Conv1d-4               [-1, 256, 3]         230,656
            Conv1d-5               [-1, 256, 1]         384,256
           Dropout-6                 [-1, 1024]               0
            Linear-7                   [-1, 14]          14,350
Total params: 3,954,974
Trainable params: 860,174
Non-trainable params: 3,094,800
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 15.09
Estimated Total Size (MB): 15.13
----------------------------------------------------------------


## Model 2

In [35]:
#pip install focal_loss_torch
#from focal_loss.focal_loss import FocalLoss
#criterion = nn.FocalLoss(gamma=.7) #maybe better to a diff func
criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
#criterion = nn.BCEWithLogitsLoss()

#optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
optimizer = torch.optim.Adadelta(model.parameters(), lr=0.05, eps=1e-06)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

In [36]:
#n_epochs = 20
def train_model(model, train_dataloader, val_dataloader, epochs=20, optimizer=optimizer, criterion=criterion):
  model.to(device)
  for epoch in range(epochs):
      model.train()
      curr_epoch_loss = []
      for batch in tqdm(train_dataloader):
          text = batch['text']
          labels = batch['labels']#.long().flatten()
          optimizer.zero_grad()
          outputs = model(text)#.flatten()
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          curr_epoch_loss.append(loss.cpu().data.numpy())
      print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss):.3f}")
      
      # Evaluation
      model.eval()
      val_loss = 0.0
      Y_shuffled, Y_preds, losses = [],[],[]
      with torch.no_grad():
          for batch in val_dataloader:
              text = batch['text']
              labels = batch['labels']#.long().flatten()
              outputs = model(text)#.flatten()
              val_loss += criterion(outputs, labels).item()  
              
              ##
              loss = criterion(outputs, labels)
              losses.append(loss.item())

              Y_shuffled.append(labels.cpu().numpy())
              Y_preds.append(outputs.cpu().numpy())

              ###
      avg_val_loss = val_loss / len(val_dataloader)

      print(f'Epoch [{epoch+1}/{epochs}], Val Loss: {avg_val_loss:.3f}')
      #print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled, Y_preds)))
  return model

In [37]:
n_epochs = 170
trained_model = train_model(model, train_loader, val_loader, n_epochs)

100%|██████████| 21/21 [00:08<00:00,  2.56it/s]


Epoch 159: curr_epoch_loss=0.125
Epoch [160/170], Val Loss: 0.316


100%|██████████| 21/21 [00:08<00:00,  2.56it/s]


Epoch 160: curr_epoch_loss=0.124
Epoch [161/170], Val Loss: 0.310


100%|██████████| 21/21 [00:08<00:00,  2.57it/s]


...

Epoch 167: curr_epoch_loss=0.118
Epoch [168/170], Val Loss: 0.311


100%|██████████| 21/21 [00:08<00:00,  2.56it/s]


Epoch 168: curr_epoch_loss=0.122
Epoch [169/170], Val Loss: 0.309


100%|██████████| 21/21 [00:08<00:00,  2.55it/s]


Epoch 169: curr_epoch_loss=0.118
Epoch [170/170], Val Loss: 0.316


In [38]:
# Model Path
PATH = '/content/drive/MyDrive/Data/trained_model.pth'

In [39]:
# Save the Model
torch.save(trained_model, PATH)

In [40]:
# Load the model
trained_model = torch.load(PATH)
print(trained_model)

CNNClassifier(
  (embedding): Embedding(10316, 300)
  (conv1): Conv1d(300, 256, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(300, 256, kernel_size=(2,), stride=(1,))
  (conv3): Conv1d(300, 256, kernel_size=(3,), stride=(1,))
  (conv4): Conv1d(300, 256, kernel_size=(5,), stride=(1,))
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=1024, out_features=14, bias=True)
)


Evaluate Model Function -

In [41]:
def eval_model(model, dataloader):
    model.eval()
    Y_pred = []
    Y_true = []
    with torch.no_grad():
        for batch in dataloader:
            text = batch['text']
            labels = batch['labels']
            pred = model(text).cpu()
            condition = pred>0.5
            pred =pred.where(condition, torch.tensor(0.0))
            condition = pred==0
            pred =pred.where(condition, torch.tensor(1.0))
            Y_pred.append(pred)
            Y_true.append(labels)
    return Y_pred, Y_true

Make Predictions on the Test Data -

In [42]:
y_pred, y_true = eval_model(trained_model, test_loader)

Evaluating the Model Performance

In [45]:
target_classes =['Obesity', 'Non.Adherence', 'Developmental.Delay.Retardation',
               'Advanced.Heart.Disease', 'Advanced.Lung.Disease',
               'Schizophrenia.and.other.Psychiatric.Disorders', 'Alcohol.Abuse',
               'Other.Substance.Abuse', 'Chronic.Pain.Fibromyalgia',
               'Chronic.Neurological.Dystrophies', 'Advanced.Cancer', 'Depression',
               'Dementia', 'Unsure']

# Collecting the Predcitions and Truth
Y_pred = torch.vstack((y_pred[0], y_pred[1], y_pred[2]))
Y_true = torch.vstack((y_true[0], y_true[1], y_true[2]))
print("Test Accuracy : {}".format(accuracy_score(Y_true.cpu(), Y_pred.cpu()), zero_division=1))
print("\nClassification Report : ")
print(classification_report(Y_true.cpu().numpy(), Y_pred.cpu(), target_names=target_classes, zero_division=1))

Test Accuracy : 0.20118343195266272

Classification Report : 
                                               precision    recall  f1-score   support

                                      Obesity       0.50      0.09      0.15        11
                                Non.Adherence       0.33      0.29      0.31        14
              Developmental.Delay.Retardation       1.00      0.33      0.50         3
                       Advanced.Heart.Disease       0.66      0.69      0.68        36
                        Advanced.Lung.Disease       0.67      0.35      0.46        17
Schizophrenia.and.other.Psychiatric.Disorders       0.76      0.34      0.47        38
                                Alcohol.Abuse       0.73      0.76      0.74        21
                        Other.Substance.Abuse       0.62      0.45      0.53        11
                    Chronic.Pain.Fibromyalgia       0.45      0.50      0.47        28
             Chronic.Neurological.Dystrophies       0.62      0.37 

In [ ]:
print(classification_report(y_true[0].cpu().numpy(), y_pred[0].cpu(), target_names=target_classes, zero_division=1))
print(classification_report(y_true[1].cpu().numpy(), y_pred[1].cpu(), target_names=target_classes, zero_division=1))
print(classification_report(y_true[2].cpu().numpy(), y_pred[2].cpu(), target_names=target_classes, zero_division=1))